#### 一. 打开FileCHannel
1. FIleChannel的获取方式有2种:
    * 输入流, 输出流
    * 从RandomAccessFile中获取
```java
RandomAccessFile aFile     = new RandomAccessFile("data/nio-data.txt", "rw");
FileChannel      inChannel = aFile.getChannel();
```    
    
#### 二. read data from FileChanel
1. `FileChannel.read()`方法, 返回的int表示读了多少字节的数据到buffer中
2. 如果`FileChannel.read()`方法返回-1, 则表示已经读到了文件末尾
```java
ByteBuffer buf = ByteBuffer.allocate(48);
int bytesRead = inChannel.read(buf);
```

#### 三. Writing Data to a FileChannel
1. ` FileChannel.write()`方法并不是一次性将buffer中的所有字节写入channel, 而是每次写入一点. 因此, ` FileChannel.write()`方法应该写在循环内部, 判断条件是`buffer.hasRemaining`
```java
buf.flip();
while(buf.hasRemaining()) {
    channel.write(buf);
}
```

#### 四. Closing a FileChannel与FileChannel Size
1. 使用`channel.close()`关闭Channel
2. 使用`channel.truncate(1024);`

#### 五. FileChannel Force
1. 使用`FileChannel.force()`方法, 强制将channel中未写入磁盘的数据写入磁盘. 
2. 为什么使用这个方法?  
   因为操作系统的设计, 往往会用一个缓存缓冲写入磁盘的数据, 调用这个方法可以强制将魂村中的数据写入磁盘, 来保证数据完整性

#### 用FileChannel完成文件复制
```java
String dir = "/Users/lj/IdeaProjects/JavaFeatures/Java NetWork/";
RandomAccessFile fromFile = new RandomAccessFile(dir+"data/from.txt","r");
FileChannel fromChannel = fromFile.getChannel();
RandomAccessFile toFile = new RandomAccessFile(dir+"data/to.txt", "rw");
FileChannel toChannel = toFile.getChannel();

long count = fromChannel.size();
System.out.println("count is:"+count);

ByteBuffer buffer = ByteBuffer.allocate(32);  // 申请一块内存缓冲
int hasRead ;
while((hasRead=fromChannel.read(buffer))!=-1){
    buffer.flip();  // 准备进入读模式
    while(buffer.hasRemaining()){
        toChannel.write(buffer);
    }
    buffer.clear(); // 准备进入写模式
}

toChannel.force(true); // 强制刷新channel中未写出的数据到磁盘
```